### ספריות

In [ ]:
import os
import sys
import pandas as pd
import geopandas as gpd
import fiona
from shapely.geometry import Point

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


### העלת משתנים להרצת הקוד

In [ ]:
path = os.getcwd()

parent = os.path.dirname(path)

software_data_folder_location = os.path.dirname(parent)

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_data_folder_location))

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

sys.path.append(software_data_folder_location)

## פונקציות

In [ ]:
from py_scripts.adding_students_to_the_population_and_age_distribution import adding_students_to_the_population_and_age_distribution
from py_scripts.adding_yeshiva_students_to_the_population_size import adding_yeshiva_students_to_the_population_size
from py_scripts.quantification_of_employees_in_jtmt_area import quantification_of_employees_in_jtmt_area
from py_scripts.unemployment_calculation import unemployment_calculation
from py_scripts.calculation_of_days import calculation_of_days
from py_scripts.successive_dispersion import successive_dispersion
from py_scripts.successive_coefficient import successive_coefficient

In [ ]:
# def drop_geo(geoDF):
#     geoDF = geoDF.drop(columns='geometry')
#     return geoDF

In [ ]:
# def double_taz_num(df):
#     dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
#     return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [ ]:
# def make_point(df):
#     df_point=df.copy()
#     df_point['centroid'] = df_point.representative_point()
#     df_point=df_point.set_geometry('centroid')
#     df_point=df_point.drop(columns=['geometry'],axis=1)
#     return df_point

In [ ]:
# def up_load_gdb(path,layer_name):
#     path='{}'.format(path)
#     layer_list=fiona.listlayers(path)
#     gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
#     return gpd_layer

In [ ]:
# def up_load_shp(path):
#     path='{}'.format(path)
#     gpd_layer=gpd.read_file(path)
#     return gpd_layer

In [ ]:
# def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
#     code_to_find=list(df_to_geoode[unique_field].unique())

#     gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

#     gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

#     gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

#     return gpd_for_geocode_in_one_taz

In [ ]:
# def up_load_df(folder_path,file_name):
    
#     path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
#     df=pd.read_excel(path_df)
#     df=df.dropna(how='all')

#     return df


### פונקציות גלובליות

In [ ]:
from functions import make_point
from functions import up_load_df
from functions import up_load_gdb
from functions import up_load_shp

In [ ]:
#העלת משתנים להרצת הקוד
# df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

# software_data_folder_location=df_inputs_outputs['location'][0]

# forecast_version_folder_location=df_inputs_outputs['location'][1]


## להריץ תלמידים

In [ ]:
%run "C:\Users\dpere\Documents\JTMT\forecast\create_forecast_basic\current\people\students_current_year_230706\students_current_year_230706.ipynb"
# %run '/people/students_current_year_230706/students_current_year_230706.ipynb'

## להעלות שכבות

In [ ]:
#העלת משתנים להרצת הקוד
# df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

# software_data_folder_location=df_inputs_outputs['location'][0]

# forecast_version_folder_location=df_inputs_outputs['location'][1]


In [ ]:
path=r'{}\background_files\EMP_KIBOLET.gdb'.format(software_data_folder_location)
EMP_kibolet=up_load_gdb(path,'EMP_kibolet')

In [ ]:
EMP_kibolet=EMP_kibolet.fillna(0)

In [ ]:
EMP_kibolet['emp_without_palestin']=EMP_kibolet['kayim_emp']-EMP_kibolet['Palestinians']

In [ ]:
emp_without_palestin_not_okev=EMP_kibolet['emp_without_palestin'].sum().item()

In [ ]:
emp_Education=taz.query('main_secto!="Palestinian"')['emp_Education'].sum().item()

## הוספת סטודנטים לאוכלוסיה ולהתפלגות גילים

In [ ]:
student_dorms=up_load_shp(r'{}\background_files\student_dorms.shp'.format(software_data_folder_location))

In [ ]:
taz=adding_students_to_the_population_and_age_distribution(student_dorms, taz)

## הוספת תלמידי ישיבה (באזורים חרדיים) אל כמות אוכלוסיה,התפלגות גילים 

In [ ]:
taz=adding_yeshiva_students_to_the_population_size(taz)

## כימות מועסקים במרחב צתאל

In [ ]:
taz=quantification_of_employees_in_jtmt_area(taz)

## חישוב אבטלה

In [ ]:
taz=unemployment_calculation(taz)

## חישוב יוממות

In [ ]:
commuting=up_load_shp(r'{}\background_files\commuting_230712.shp'.format(software_data_folder_location))

In [ ]:
taz['commuting']=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],commuting[['commuting', 'geometry']]).set_index('Taz_num')['commuting']/100

taz['pop_emp_employed_out_of_jtmt_area']=taz['pop_emp_employed']*taz['commuting']

emp_left_jtmt_area=taz['pop_emp_employed_out_of_jtmt_area'].sum().item()

round(emp_left_jtmt_area,-3)

emp_from_jtmt_area=taz['pop_emp_employed'].sum().item()-emp_left_jtmt_area

# יוממות פנימה מחוץ למרחב
emp_in_jtmt_area=emp_from_jtmt_area*1.07

emp_in_jtmt_area_without_mobile=emp_in_jtmt_area*0.95 #הפחחת עובדים ניידים

round(emp_in_jtmt_area_without_mobile,-3)

emp_okev=emp_in_jtmt_area_without_mobile-emp_Education-emp_without_palestin_not_okev

# taz=calculation_of_days(taz, commuting)

## פיזור עוקב

In [ ]:
taz=successive_dispersion(EMP_kibolet, taz)

## מקדם עוקב

In [ ]:
taz=successive_coefficient(taz, emp_okev, software_data_folder_location)

## פיצול שכבת לא עוקב לאזורי תנועה

In [ ]:
EMP_kibolet['emp_area']=EMP_kibolet.area

In [ ]:
EMP_kibolet['ID']=EMP_kibolet.index

In [ ]:
EMP_kibolet_by_taz=gpd.overlay(taz.reset_index()[['Taz_num','taz_area','geometry']],EMP_kibolet)

In [ ]:
EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

In [ ]:
EMP_kibolet_by_taz['pre_small_area_emp']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['emp_area']

In [ ]:
EMP_kibolet_by_taz['pre_small_area_taz']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['taz_area']

In [ ]:
EMP_kibolet_by_taz=EMP_kibolet_by_taz.query('(pre_small_area_taz >0.7) | (pre_small_area_emp >0.1)').drop(columns='geometry_buff')

In [ ]:
EMP_kibolet_by_taz=EMP_kibolet_by_taz.set_index('ID')

In [ ]:
EMP_kibolet_by_taz['id_area_for_pre']=EMP_kibolet_by_taz.reset_index().groupby(by='ID')['small_area'].sum()

In [ ]:
EMP_kibolet_by_taz[['id_area_for_pre']]

In [ ]:
EMP_kibolet_by_taz['prec_from_id']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['id_area_for_pre']

In [ ]:
EMP_kibolet_by_taz['emp_without_palestin_in_taz']=EMP_kibolet_by_taz['emp_without_palestin']*EMP_kibolet_by_taz['prec_from_id']

In [ ]:
taz['emp_not_okev']=drop_geo(EMP_kibolet_by_taz).pivot_table(index='Taz_num',aggfunc=sum)[['emp_without_palestin_in_taz']]

In [ ]:
taz=taz.fillna(0)

In [ ]:
taz['total_emp']=taz['emp_not_okev']+taz['emp_okev']+taz['emp_Education']

In [ ]:
taz.query('main_secto!="Palestinian"')['total_emp'].sum()-emp_in_jtmt_area_without_mobile

## תוצאות

In [ ]:
taz.groupby(by='zonetype')[['total_emp']].sum()#.loc[['Jerusalem','Judea and Samaria']]

In [ ]:
taz.groupby(by='Muni_Heb')[['total_emp']].sum().loc[['ירושלים','בית שמש','ביתר עילית','מודיעין - מכבים - רעות','מעלה אדומים','מודיעין עילית']]

In [ ]:
taz.groupby(by=['Muni_Heb','main_secto'])[['total_emp']].sum().loc[['ירושלים']]

## התפלגות ענפי תעסוקה

In [ ]:
emp_category_type=up_load_shp(r'{}\background_files\emp_category_type.shp'.format(software_data_folder_location)).fillna(0)

In [ ]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

In [ ]:
emp_category_type[col]=emp_category_type[col]/100

In [ ]:
emp_category_type=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],emp_category_type).set_index('Taz_num')

In [ ]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

In [ ]:
taz.join(emp_category_type[col]).query('(emp_not_okev>0 | emp_okev>0 )& agri.isna() & main_secto!="Palestinian" ')

In [ ]:
taz=taz.join(emp_category_type[col]).fillna(0)

In [ ]:
for i in col:
    taz['{}'.format(i)]=taz['{}'.format(i)]*(taz['emp_not_okev']+taz['emp_okev'])

In [ ]:
taz['check_emp_sum']=taz[col].sum(axis=1)-(taz['emp_not_okev']+taz['emp_okev'])

In [ ]:
taz.query('check_emp_sum>10 |check_emp_sum<-10')

## הוספת סטודנטים אל כמות משקי בית

In [ ]:
taz=taz.fillna(0)

In [ ]:
taz['hh']=taz['aprt_20']+taz['student_dorms']/1.5

## הוספת תלמידי ישיבה (באזורים לא חרדיים) אל כמות אוכלוסיה,התפלגות גילים ומשקי בית (לכלל תלמידי הישיבה)

In [ ]:
taz['hh']=taz['hh']+taz['yeshiva_dorms_pop_sum']

In [ ]:
taz.loc[taz['main_secto']!="U_Orthodox",'pop']=taz['pop']+taz['yeshiva_dorms_pop_sum']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_15']=taz['pop_15']+taz['yeshiva_dorms_pop_15']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_20']=taz['pop_20']+taz['yeshiva_dorms_pop_20']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_25']=taz['pop_25']+taz['yeshiva_dorms_pop_25']

## הוספת אוכלוסיה פלסטינאית

In [ ]:
path=r'{}\background_files'.format(software_data_folder_location)
taz_demo_pls_2020=up_load_df(path,'taz_demo_pls_2020_and_pre_growth_till_2050')[['Taz_num','pop_2020']].set_index('Taz_num')

In [ ]:
col=['pop',
'pop_0',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_5',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'student',
'uni_students',
'student_yeshiva_and_kollim',
'emp_Education',
'pop_emp_employed',
'total_emp',
'agri',
'Indus',
'Com_hotel',
'Business',
'Public',
'hh']

In [ ]:
taz.loc[taz['main_secto']=='Palestinian',col]=0

In [ ]:
taz.loc[taz['main_secto']=='Palestinian','pop']=taz['pop']+taz_demo_pls_2020['pop_2020']

## ייצוא לבקרה

In [ ]:
taz=taz.reset_index()

In [ ]:
pre_yeshiva_from_20_30_age=taz[['student_yeshiva_with_add_from_old']].sum().sum()/taz.query('jew==1')[['pop_20_just_from_aprt','pop_25_just_from_aprt']].sum().sum()

In [ ]:
pre_yeshiva_from_20_30_age

In [ ]:
taz.main_secto.unique()

In [ ]:
pre_uni_from_20_30_age=taz[['uni_students']].sum().sum()/taz.query('main_secto!="Palestinian"')[['pop_20_just_from_aprt','pop_25_just_from_aprt']].sum().sum()

In [ ]:
pre_uni_from_20_30_age

In [ ]:
list(taz)

In [ ]:
col=['Taz_num','Agg_taz_nu','Taz_name','Muni_Heb','jeru_metro',
 'zonetype',
 'in_jerusal','main_secto','aprt_20','student_dorms','student_yeshiva_with_add_from_old','hh','pop_without_dorms_yeshiva',
 'pop', 'hh_size','pop_emp_employed',
 'commuting','student_toddlers',
 'student_gov','cbs_muni_student_left_by_pre_of_demand_left','cbs_muni_student_left_by_pre_of_demand','student_chardi_not_gov',
 'student_arab_not_gov','student','uni_students',
 'student_yeshiva','yeshiva_dorms_pop_15',
 'yeshiva_dorms_pop_20',
 'yeshiva_dorms_pop_25',
 'yeshiva_dorms_pop_sum',
 'add_from_old_student_yeshiva',
 'add_from_kollim_demand', 'emp_from_uni_student','emp_from_Yeshiva_student','emp_from_student',
 'emp_Education','okev',
 'emp_okev',
 'emp_not_okev','total_emp',
 'agri',
 'Indus',
 'Com_hotel',
 'Business',
 'Public']

In [ ]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [ ]:
taz[col].to_excel(r'{}\2020_jtmt_forcast_full_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

## ייצוא בפורמט של מודלים

In [ ]:
taz['yosh']=0

In [ ]:
taz.loc[taz['zonetype']=='Judea and Samaria','yosh']=1

In [ ]:
taz['jerusalem_city']=0

In [ ]:
taz.loc[taz['in_jerusal']=='yes','jerusalem_city']=1

In [ ]:
taz=taz.reset_index()

In [ ]:
col=['Taz_num',
'yosh',
'jeru_metro',
'jerusalem_city',
'main_secto',
'hh',
'pop',
'pop_0',
'pop_5',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'total_emp',
'Indus',
'Com_hotel',
'Business',
'Public',
'emp_Education',
'agri',
'student',
'uni_students',
'student_yeshiva_and_kollim',
'pop_emp_employed']

In [ ]:
col_new_name=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'sector',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

In [ ]:
taz=taz[col]

In [ ]:
taz.columns=col_new_name

In [ ]:
taz.to_excel(r'{}\forecast_2020_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

## בקרות לוגיות

In [ ]:
taz.loc[taz['TAZ']<=0]

In [ ]:
taz.yosh.unique()

In [ ]:
taz.in_jerusalem_metropolin.unique()

In [ ]:
taz.jerusalem_city.unique()

In [ ]:
taz.sector.unique()

In [ ]:
col=['hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

In [ ]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]<0]['TAZ'])

In [ ]:
taz_to_check

In [ ]:
taz.loc[~(taz['hh_total']<=taz['pop'])]

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['hh_total']>0].loc[taz['pop']<=0]

In [ ]:
col=['age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0].query('TAZ==511')#.to_excel('delet.xlsx')#[col].sum(axis=1)

In [ ]:
len(taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0])

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop']>0].loc[taz['hh_total']<=0]

In [ ]:
col=[
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up']

In [ ]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['pop']<=0]['TAZ'])
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['hh_total']<=0]['TAZ'])

In [ ]:
taz_to_check

In [ ]:
col=['indus',
'com_hotel',
'business',
'public',
'education',
'agri']

In [ ]:
taz['emp_check']=round(taz[col].sum(axis=1)-taz['emp_tot'])

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['emp_check']!=0]

In [ ]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['emp_tot']<=0]['TAZ'])

In [ ]:
taz_to_check

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_emp_employed']>0].loc[taz['pop']<=0]